In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np

In [2]:
train_path= r'C:\Workstation\Chest_Cancer\train'
validation_path= r'C:\Workstation\Chest_Cancer\valid'
test_path= r'C:\Workstation\Chest_Cancer\test'

In [3]:
resize_tuple= (64,64)

In [4]:
from PIL import Image
img= Image.open(r'C:\Workstation\Chest_Cancer\test\squamous.cell.carcinoma\000117 (3).png')
print(img.size)
img= transforms.functional.resize(img, resize_tuple)
img= transforms.functional.adjust_sharpness(img, 4)
img.show()

(429, 353)


In [4]:
train_transform= transforms.Compose(
    [
        transforms.Resize(resize_tuple),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomAutocontrast(p=0.7),
        transforms.RandomAdjustSharpness(8, p=0.8),
        transforms.ToTensor()
    ]
)

In [5]:
validation_transform= transforms.Compose(
    [
        transforms.Resize(resize_tuple),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomAutocontrast(p=0.7),
        transforms.RandomAdjustSharpness(8, p=0.8),
        transforms.ToTensor()
    ]
)

In [6]:
test_transform= transforms.Compose(
    [
        transforms.Resize(resize_tuple),
        transforms.ToTensor()
    ]
)

In [7]:
train_dataset= torchvision.datasets.ImageFolder(train_path, transform= train_transform)
validation_dataset= torchvision.datasets.ImageFolder(validation_path, transform= validation_transform)
test_dataset= torchvision.datasets.ImageFolder(test_path, transform= test_transform)

In [8]:
from torch.utils.data import DataLoader

In [9]:
train_load= DataLoader(train_dataset, batch_size= 64, shuffle= True)
validation_load= DataLoader(validation_dataset, batch_size= 64, shuffle= True)
test_load= DataLoader(test_dataset, batch_size= 64, shuffle= False)

In [10]:
import torch.nn as nn

Implementing a simple CNN model

In [104]:
class CNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1= nn.Conv2d(3,32,2,stride=2)
        self.pool1= nn.MaxPool2d(4,2)
        self.conv2= nn.Conv2d(32,64,5)
        self.pool2= nn.AvgPool2d(3,1)
        self.fc1= nn.Linear(9*9*64, 64, bias= True)
        self.fc2= nn.Linear(64,24, bias=True)
        self.fc3= nn.Linear(24,4)
        self.relu= nn.ReLU()
        self.out= nn.Softmax(dim=1)
    def forward(self, image):
        out= self.relu(self.conv1(image))
        out= self.relu(self.pool1(out))
        out= self.relu(self.conv2(out))
        out= self.relu(self.pool2(out))
        out= torch.flatten(out,1)
        out= self.relu(self.fc1(out))
        out= self.relu(self.fc2(out))
        out= self.out(self.fc3(out))
        return out

Implementing an inception v2 here.

In [78]:
class CNN_inception_v2(nn.Module):                                      
    def __init__(self) -> None:
        super().__init__()
        self.conv= nn.Conv2d(3, 64, 5, stride= 1)          #In_channel is 3 cuz its an RGB image. out_channel signifies number of filters used. In this case its 10
        self.pool= nn.MaxPool2d(4,2)
        self.conv1dim= nn.Conv2d(64, 32, 1)
        self.conv3dim= nn.Conv2d(32, 128, 3, padding= 1)
        self.conv5dim= nn.Conv2d(32, 64, 5, padding= 2) 
        self.poolincep= nn.MaxPool2d(3,1,padding= 1) 
        self.convpin= nn.Conv2d(256,256, 3, 2)
        self.fc1= nn.Linear(14*14*256, 256)      
        self.drop1= nn.Dropout(p= 0.2)
        self.fc2= nn.Linear(256,64)
        self.drop2= nn.Dropout(p= 0.1)
        self.fc3= nn.Linear(64, 16)
        self.drop3= nn.Dropout(p= 0.1)
        self.fc4= nn.Linear(16,4, bias=True)
        self.relu= nn.ReLU()
        self.sig= nn.Sigmoid()
        self.soft= nn.Softmax(dim=1)
    def forward(self, image):
        out= self.relu(self.conv(image))
        out= self.relu(self.pool(out))
        out1= self.conv1dim(out)
        out3= self.conv3dim(out1)
        out5= self.conv5dim(out1)
        outpool= self.conv1dim(self.poolincep(out))
        #print(f'out3 {out3.shape}, out5 {out5.shape}, outpool {outpool.shape} ')
        out= torch.cat((out1,out3,out5,outpool), dim=1)             #We use dim=1 for channel wise concat
        #print(out.shape)
        out= self.relu(self.convpin(out))
        out= torch.flatten(out,1)                                   #start dim is 1 cuz dim 0 is the batch size
        out= self.relu(self.fc1(out))
        out= self.drop1(out)
        out= self.relu(self.fc2(out))
        out= self.drop2(out)
        out= self.relu(self.fc3(out))
        out= self.soft(self.fc4(out))
        return out

In [12]:
import torch.nn.functional as F

In [47]:
import torch.optim as optim

In [13]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [108]:
model_in= CNN_inception_v2()
model_in.to(device)
criterion= nn.CrossEntropyLoss()
optimizer_incep= optim.Adam(model_in.parameters(), lr= 0.0005, weight_decay= 0.01)

In [109]:
model_cnn= CNN()
model_cnn.to(device)
criterion= nn.CrossEntropyLoss()
optimizer_cnn= optim.Adam(model_cnn.parameters(), lr= 0.001, weight_decay= 0.01) 

In [111]:
def train_model(model, train_load, epochs, optimizer):
    for i in range(epochs):
        running_loss=0
        for image, labels in train_load:
            optimizer.zero_grad()
            #label_enc= F.one_hot(labels)
            pred= model(image.to(device))
            loss= criterion(pred, labels.to(device))
            loss.backward()
            optimizer.step()
            running_loss+=loss.item()
        print(running_loss/len(train_load))
    return model

In [99]:
model_in= train_model(model_in,train_load,epochs=80, optimizer= optimizer_incep)

1.3804609656333924
1.371563971042633
1.371424102783203
1.372449231147766
1.3730290532112122
1.3737617492675782
1.375681495666504
1.3689804077148438
1.3652426958084107
1.3646875262260436
1.3624593257904052
1.33995840549469
1.3054252028465272
1.3378654837608337
1.3170722842216491
1.3040712833404542
1.2911670207977295
1.2834850430488587
1.2779454946517945
1.2855224370956422
1.2810354709625245
1.274007225036621
1.267010235786438
1.2772723078727721
1.2609099864959716
1.2538387775421143
1.2668240070343018
1.2548731446266175
1.2558371782302857
1.2557998299598694
1.2664166927337646
1.2554449796676637
1.2593890309333802
1.2423535466194153
1.2454298615455628
1.241891360282898
1.2555890321731566
1.2479472875595092
1.2480343222618102
1.2453110694885254
1.2344878673553468
1.2413658618927002
1.22824684381485
1.2233487606048583
1.2248245120048522
1.237891936302185
1.2225700974464417
1.2196126222610473
1.2038466453552246
1.2204222202301025
1.2263949155807494
1.214055562019348
1.2112244844436646
1.1999

In [17]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, recall_score, precision_score

In [100]:
model_in.cpu()

CNN_inception_v2(
  (conv): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1dim): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (conv3dim): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5dim): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (poolincep): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
  (convpin): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2))
  (fc1): Linear(in_features=50176, out_features=256, bias=True)
  (drop1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (drop2): Dropout(p=0.1, inplace=False)
  (fc3): Linear(in_features=64, out_features=16, bias=True)
  (drop3): Dropout(p=0.1, inplace=False)
  (fc4): Linear(in_features=16, out_features=4, bias=True)
  (relu): ReLU()
  (sig): Sigmoid()
  (soft): Softmax(dim=1)
)

In [39]:
def accuracy_test(data_load, model):
    for image, label in data_load:
        pred= model(image)
        label= F.one_hot(label)
        print(accuracy_score(np.argmax(pred.detach().numpy(),axis=1), np.argmax(label.detach().numpy(), axis=1)))

In [101]:
accuracy_test(train_load, model_in)

0.6875
0.640625
0.546875
0.515625
0.59375
0.609375
0.765625
0.5625
0.5625
0.5675675675675675


In [102]:
accuracy_test(validation_load, model_in)

0.53125
0.5


In [103]:
accuracy_test(test_load, model_in)

0.015625
0.03125
0.203125
0.640625
0.5254237288135594


In [106]:
del model_in
torch.cuda.empty_cache()

In [113]:
model_cnn= train_model(model_cnn, train_load, epochs= 50,optimizer= optimizer_cnn)

1.3805128574371337
1.3739952087402343
1.3691155791282654
1.370361065864563
1.372713530063629
1.3657234072685243
1.3554505944252013
1.3682892799377442
1.3565939903259276
1.3519548416137694
1.3281307101249695
1.3167996048927306
1.3041876673698425
1.2800137639045714
1.2771039366722108
1.274445855617523
1.2732572913169862
1.2746810078620912
1.2692505359649657
1.2620531797409058
1.2671003818511963
1.27033371925354
1.2549307227134705
1.2549286961555481
1.2463118433952332
1.2586859583854675
1.2499377131462097
1.2493787288665772
1.2502614498138427
1.2508395433425903
1.2476540446281432
1.2444623827934265
1.2403372883796693
1.2224214315414428
1.2309369087219237
1.2264047384262085
1.2235989212989806
1.2203340530395508
1.2249328017234802
1.2398862838745117
1.2198015570640564
1.2152350068092346
1.2066137790679932
1.2438100099563598
1.2513516902923585
1.225580060482025
1.209929347038269
1.2102306604385376
1.2381454706192017
1.2359827280044555


In [115]:
model_cnn.cpu()

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(2, 2), stride=(2, 2))
  (pool1): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (pool2): AvgPool2d(kernel_size=3, stride=1, padding=0)
  (fc1): Linear(in_features=5184, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=24, bias=True)
  (fc3): Linear(in_features=24, out_features=4, bias=True)
  (relu): ReLU()
  (out): Softmax(dim=1)
)

In [116]:
accuracy_test(train_load, model_cnn)

0.453125
0.53125
0.5
0.484375
0.5
0.59375
0.578125
0.453125
0.359375
0.5945945945945946


In [117]:
accuracy_test(validation_load, model_cnn)

0.40625
0.375


In [118]:
accuracy_test(test_load, model_cnn)

0.828125
0.609375
0.203125
0.59375
0.23728813559322035


In [119]:
del model_cnn
torch.cuda.empty_cache()